In [72]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy

In [73]:
def taxdf(calc, namelist, varlist):
    calcu=copy.deepcopy(calc)
    tdf=pd.DataFrame()
    for v in range (0, len(namelist)):
        tdf[namelist[v]]=getattr(calcu.records,varlist[v])
    return tdf

In [74]:
path_name = 'C:\\Users\\derrick.choe\\Tax-Calculator\\taxcalc\\reforms\\TCJA_Reconciliation.json'
jfile = open(path_name, 'r')
jtext = jfile.read()
jfile.close()
reform = tc.Calculator._read_json_policy_reform_text(jtext, {}, {})
reform_noctc = tc.Calculator._read_json_policy_reform_text(jtext, {}, {})

for y in range(2017,2027):
    if y in reform_noctc.keys():
        reform_noctc[y]['_CTC_c'] = [1000]
        reform_noctc[y]['_DependentCredit_Child_c'] = [0]
        reform_noctc[y]['_ACTC_Income_thd'] = [3000]
        reform_noctc[y]['_CTC_ps'] = [[75000, 110000, 55000, 75000, 75000]]

# breform = {2018:{'_BE_sub': [0.25], '_BE_subinc_wrt_earnings': [True]}}

# Create calculator for TCJA Reconciliation plan, minus refundable child tax credit

pol_noctc = tc.Policy()
pol_noctc.implement_reform(reform_noctc)

calc = tc.Calculator(policy = pol_noctc, records = tc.Records())
calc.advance_to_year(2018)
calc.calc_all()

# Create calculator for TCJA Reconciliation plan

pol = tc.Policy()
pol.implement_reform(reform)

calc1 = tc.Calculator(records = tc.Records(), policy = pol)
calc1.advance_to_year(2018)
calc1.calc_all()

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [76]:
nlist = ['agi','std_amt','itm_amt','iitax','combined','payroll', 's006','mars','n24','xtot']
vlist = ['c00100', 'standard', 'c04470', 'iitax','combined','payrolltax','s006','MARS', 'n24', 'XTOT']
calcdf = taxdf(calc=calc, namelist=nlist, varlist=vlist)
calcdf1 = taxdf(calc=calc1, namelist = nlist, varlist = vlist)

In [93]:
# GET TOTAL MONEY SPENT ON CHILD TAX REFUNDS
calcdf['w_iitaxdiff']=(calcdf['iitax']-calcdf1['iitax'])*calcdf['s006']
calcdf['w_iitax'] = calcdf1['iitax'] * calcdf1['s006']
calcdf['w_combined']=(calcdf['combined']-calcdf1['combined'])*calcdf['s006']
calcdf['iitaxdiff']=calcdf['iitax']-calcdf1['iitax']
calcdf_sp=copy.deepcopy(calcdf[calcdf['iitax']<0])
single_kids=calcdf_sp[( (calcdf_sp['mars']==4))]['s006'].sum()
single_kids1=calcdf_sp[( (calcdf_sp['mars']==1) | (calcdf_sp['mars']==4))]['s006'].sum()
total=calcdf_sp['s006'].sum()
# print("Fraction of (Refund Receiving) Beneficiaries that are Head of Household with Kids: " +str(single_kids/total))
# print("Number of (Refund Receiving) Beneficiaries that are Head of Household with Kids: " +str(round(single_kids)))
print("Fraction of (Refund Receiving) Beneficiaries that are Single or Head of Household with Kids: " +str(single_kids1/total))
print("Number of (Refund Receiving) Beneficiaries that are Single or Head of Household with Kids: " +str(round(single_kids1)))
print("Number of (Refund Receiving) Beneficiaries: " +str(round(total)))
print("Amount of Revenue Going towards Taxpayers Already Receiving Refunds: " + str(round(calcdf_sp['w_iitaxdiff'].sum())))
print("Amount of Revenue for Entire Reform: " + str(round(calcdf['w_combined'].sum())))
print('Amount of Revenue Going Towards Single Parent Beneficiaries: ' + str(round(calcdf_sp[( (calcdf_sp['mars']==1) | (calcdf_sp['mars']==4)) ]['w_iitaxdiff'].sum()
)))
calcdf_sp[( (calcdf_sp['mars']==1) | (calcdf_sp['mars']==4))]['w_iitax'].sum()

Fraction of (Refund Receiving) Beneficiaries that are Single or Head of Household with Kids: 0.773047002617
Number of (Refund Receiving) Beneficiaries that are Single or Head of Household with Kids: 24997568.0
Number of (Refund Receiving) Beneficiaries: 32336414.0
Amount of Revenue Going towards Taxpayers Already Receiving Refunds: 15345656793.0
Amount of Revenue for Entire Reform: 66952583983.0
Amount of Revenue Going Towards Single Parent Beneficiaries: 10008230773.0


-75529279665.266907